# Text Classification and Explanation

## Dataset Download

In [2]:
DATASET = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
DATADIR = './movie_review'

In [3]:
import requests
import os
from tqdm import tqdm
import tarfile
import random
import pandas as pd
import re
import numpy as np
pd.set_option('display.max_colwidth', None)


In [4]:
def download_dataset(url, datadir, overwrite=False):
    if not os.path.exists(datadir):
        os.makedirs(datadir)
    elif not overwrite:
        return
    res = requests.get(url, allow_redirects=True)
    file_name = os.path.basename(url)
    with tqdm.wrapattr(open(f"{datadir}/{file_name}", "wb"), "write",
                   miniters=1, desc=file_name,
                   total=int(res.headers.get('content-length', 0))) as datafile:
        for chunk in res.iter_content(chunk_size=4096):
            datafile.write(chunk)

In [5]:
download_dataset(DATASET, DATADIR, overwrite=False)

In [6]:
def extract_dataset(datadir, overwrite=False):
    if os.path.exists(f"{datadir}/aclImdb") and not overwrite:
        return
    with tarfile.open("{}/aclImdb_v1.tar.gz".format(datadir)) as movie_tar:
        movie_tar.extractall(datadir)


In [7]:
extract_dataset(DATADIR)

## Load Dataset

In [8]:
class MovieDataset:
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.pos_files = [(os.path.join(self.data_dir,'aclImdb','train','pos',file),1) for file in os.listdir(os.path.join(self.data_dir,'aclImdb','train','pos'))]
        self.neg_files = [(os.path.join(self.data_dir,'aclImdb','train','neg',file),0) for file in os.listdir(os.path.join(self.data_dir,'aclImdb','train','neg'))]
        self.all_files = self.pos_files + self.neg_files
        np.random.seed(42)
        np.random.shuffle(self.all_files)
        
    def load(self, lower=False):
        data = []
        for file, cat in tqdm(self.all_files):
            with open(file) as f:
                content = f.read().lower() if lower else f.read()
                content = content.replace("\n","")
                content = re.sub('[`,.\!\'\"\-!@#\$%^&*()_\+=~<>]','',content)
                data.append((content, cat))
        return pd.DataFrame(data, columns=["review", "sentiment"])

In [9]:
df = MovieDataset(DATADIR).load(lower=True)

100%|██████████| 25000/25000 [00:06<00:00, 3799.84it/s]


In [10]:
df.head()

,review,sentiment
0,great little thriller i was expecting some type of silly horror movie but what i got was tight short thriller that waste none of our time mostof these movies we have to get into the back characters stories so we will either feel sympathy for them or hatred when people start getting killed o such foolishness here yes you see a few characters but they really only interact with the principals such as the husband wife at the motel whose room was canceled we saw them so we could just how efficient the lisa character was and how inefficient the new hotel clerk was we see the little girl simply because she will have a very small but important role later in the movie when all heck breaks loose the flight atrendants because we need on in particular to move the plot ahead the bad guy in particular needs her in the beginning of the flight the rude guy in the airport was important to the movie too the only 2 characters that were just 5 liners with no use to the plot were the two young guys on the plane that was clever because i thought they would have something to do with plot from the first scene to the last the woman character a young hotel executive named lisa is in charge even when jackson shows his true colors she doesnt panic she thinks what she can do to stall time any other movie the smart executive women would be acting like idiots but not this one it was a very short movie and i was waiting for the usual plot devices to kick in because the movie seemed to be coming to its conclusion fast thankfuly none of them were used the new hotel clerk did not do the usual called and told her what to do which is panic drop the phone and run out of the hotel without saying anything or question your boss and tell her she had to much to drink and just dismiss her was craven should do more of these types of movies also one last comment brian cox is in the movie but i had not one clue who he was i had to come over here to see that he is lisas father he is completely unrecognizable,1
1,nothing could have saved this movie not even supermanbr /br /ten years ago the special effects would have been amazing better directing might have gotten some more feeling and better performances out of the actors but nothing but feeding the script to a dragon could have fixed it plot holes bad lines terrible pacing endless replaying of the same shots of a cgi dragon stomping through hallways ughbr /br /avoid this one at all costsbr /br /,0
2,this was a good movie it wasnt your typical war flick but something a bit different this movie showed us recruits in training before the war and not actually fighting a war this film is one of the more realist views on war and the army than most other films like this made colin farrel did a great job at portraying an army recruit and clifton collins jr and also matthew davis contributed reasonably well seeing colin farrel move from b grade to a grade in a few short years you would never thought it would happen i will also add that the makers of tigerland did a great job at filming a movie in the year 2000 and making it look like 1970 was a good touch its good to see talent used wisely and i hope to see this same talent again in the near future,1
3,from the pen of richard condon the manchurian candidate 1962 comes this muddled tale of political intrigue and assassination the story told in almost comic book fashion is difficult to swallow allstar cast considered this poor effort is not entirely the fault of the cast and crew: the novel was replete with the same shortcomings it seems as though at times the story is actually mocking the more sincere effort put forth in manchurian candidate a disappointment on all counts,0
4,i suppose that today this film has relevance because it was an early sofia loren film she was 19 years old when the film was made in 1953br /br /i viewed this film because i wanted to see some of sofia lorens early work i was surprised when she came on camera having had her skin bronzed over 

## Model Training

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
from nltk import word_tokenize

In [13]:
tf_vectorizer = TfidfVectorizer(stop_words='english', tokenizer=word_tokenize, analyzer='word', ngram_range=(1,3))

In [14]:
df['review_vector'] = tf_vectorizer.fit_transform(df['review'])

In [15]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [16]:
trainx_orig, testx_orig, trainy, testy = train_test_split(df['review'], df['sentiment'].values, test_size=0.15, random_state=42)

In [17]:
trainx = tf_vectorizer.transform(trainx_orig)
testx = tf_vectorizer.transform(testx_orig)

In [18]:
train_data = xgb.DMatrix(trainx, label=trainy)
test_data = xgb.DMatrix(testx, label=testy)

In [19]:
# trainx.dtypes

In [20]:
params = {'booster': 'dart', 'lambda': 4.3445606132944827e-07, 'alpha': 1.6837667313044715e-05, 'max_depth': 9, 'eta': 0.9995441483545477, 'gamma': 8.685419715245789e-08, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 2.53345870065096e-08, 'skip_drop': 7.839688797876259e-07}

In [21]:
xgb_model = xgb.train(params, train_data)

In [22]:
predictions = xgb_model.predict(test_data)

In [23]:
predict_classes = np.where(predictions>=0.5, 1, 0)

In [24]:
from sklearn.metrics import classification_report, accuracy_score, auc, roc_curve, roc_auc_score

In [25]:
print(classification_report(testy, predict_classes))

              precision    recall  f1-score   support

           0       0.78      0.74      0.76      1850
           1       0.76      0.79      0.77      1900

    accuracy                           0.77      3750
   macro avg       0.77      0.77      0.77      3750
weighted avg       0.77      0.77      0.77      3750



### Using Optuna

In [276]:
import optuna

In [277]:
def objective(trial):
#     trainx, testx, trainy, testy = train_test_split(tf_vectorizer.transform(df['review']), df['sentiment'].values, test_size=0.25)
#     dtrain = xgb.DMatrix(train_x, label=train_y)
#     dvalid = xgb.DMatrix(testx, label=testy)

    param = {
        "verbosity": 0,
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "booster": trial.suggest_categorical("booster", ["gbtree",  "dart"]),
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
    }

    if param["booster"] == "gbtree" or param["booster"] == "dart":
        param["max_depth"] = trial.suggest_int("max_depth", 1, 9)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    # Add a callback for pruning.
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-auc")
    bst = xgb.train(param, train_data, evals=[(test_data, "validation")], callbacks=[pruning_callback])
    preds = bst.predict(test_data)
    pred_labels = np.rint(preds)
    accuracy = accuracy_score(testy, pred_labels)
    return accuracy


In [278]:
study = optuna.create_study(
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=5), direction="maximize"
    )

[I 2020-12-25 21:48:20,937] A new study created in memory with name: no-name-48a88ea7-470a-40ff-af1c-a213cc3ce5c2


In [279]:
study.optimize(objective, n_trials=100)

/Users/mayorao/opt/miniconda3/envs/bertnlp/lib/python3.8/site-packages/xgboost/training.py:19: UserWarning: Old style callback is deprecated.  See: https://xgboost.readthedocs.io/en/latest/python/callbacks.html
  warnings.warn(f'Old style callback is deprecated.  See: {link}', UserWarning)


[0]	validation-auc:0.76554
[1]	validation-auc:0.77135
[2]	validation-auc:0.77156
[3]	validation-auc:0.77207
[4]	validation-auc:0.77303
[5]	validation-auc:0.77187
[6]	validation-auc:0.77355
[7]	validation-auc:0.77352
[8]	validation-auc:0.77419
[9]	validation-auc:0.77517


[I 2020-12-25 21:49:27,213] Trial 0 finished with value: 0.7224 and parameters: {'booster': 'gbtree', 'lambda': 4.874900261502898e-06, 'alpha': 1.592256356468614e-08, 'max_depth': 9, 'eta': 0.005727899704604566, 'gamma': 0.0016107186459368345, 'grow_policy': 'lossguide'}. Best is trial 0 with value: 0.7224.


[0]	validation-auc:0.75464
[1]	validation-auc:0.75527
[2]	validation-auc:0.75554
[3]	validation-auc:0.75557
[4]	validation-auc:0.75471
[5]	validation-auc:0.75457
[6]	validation-auc:0.75617
[7]	validation-auc:0.75637
[8]	validation-auc:0.75664
[9]	validation-auc:0.75658


[I 2020-12-25 21:50:07,929] Trial 1 finished with value: 0.7048 and parameters: {'booster': 'dart', 'lambda': 0.0030203294358463395, 'alpha': 4.969790026399989e-05, 'max_depth': 7, 'eta': 0.0027503289438937876, 'gamma': 5.3524366554055905e-08, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 1.403438181595057e-07, 'skip_drop': 2.6423317479540606e-05}. Best is trial 0 with value: 0.7224.


...

[I 2020-12-25 22:41:31,560] Trial 97 pruned. Trial was pruned at iteration 5.


[0]	validation-auc:0.76554
[1]	validation-auc:0.79991
[2]	validation-auc:0.82020
[3]	validation-auc:0.83477
[4]	validation-auc:0.84425
[5]	validation-auc:0.85223
[6]	validation-auc:0.85665
[7]	validation-auc:0.86181
[8]	validation-auc:0.86513
[9]	validation-auc:0.87021


[I 2020-12-25 22:42:13,419] Trial 98 finished with value: 0.7936 and parameters: {'booster': 'dart', 'lambda': 5.133846226368338e-07, 'alpha': 2.1280522090854086e-06, 'max_depth': 9, 'eta': 0.6785089023311506, 'gamma': 1.0958973019391491e-07, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 3.148130582988392e-08, 'skip_drop': 2.3680417404120534e-06}. Best is trial 75 with value: 0.8018666666666666.


[0]	validation-auc:0.76554
[1]	validation-auc:0.80046
[2]	validation-auc:0.81966
[3]	validation-auc:0.83230
[4]	validation-auc:0.84454
[5]	validation-auc:0.85043
[6]	validation-auc:0.85670
[7]	validation-auc:0.85948
[8]	validation-auc:0.86291
[9]	validation-auc:0.86667


[I 2020-12-25 22:43:08,224] Trial 99 finished with value: 0.7890666666666667 and parameters: {'booster': 'gbtree', 'lambda': 4.041983391507205e-07, 'alpha': 2.3059667320203534e-06, 'max_depth': 9, 'eta': 0.6541236526128846, 'gamma': 2.316274291334577e-07, 'grow_policy': 'depthwise'}. Best is trial 75 with value: 0.8018666666666666.


In [280]:
print(study.best_trial.params)

{'booster': 'dart', 'lambda': 4.3445606132944827e-07, 'alpha': 1.6837667313044715e-05, 'max_depth': 9, 'eta': 0.9995441483545477, 'gamma': 8.685419715245789e-08, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 2.53345870065096e-08, 'skip_drop': 7.839688797876259e-07}


#### Best Param
{'booster': 'dart', 'lambda': 4.3445606132944827e-07, 'alpha': 1.6837667313044715e-05, 'max_depth': 9, 'eta': 0.9995441483545477, 'gamma': 8.685419715245789e-08, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 2.53345870065096e-08, 'skip_drop': 7.839688797876259e-07}

In [281]:
bst = xgb.train(study.best_trial.params, train_data)

In [282]:
new_preds = np.where(bst.predict(test_data)>=0.5,1,0)

In [283]:
print(classification_report(testy, new_preds))

              precision    recall  f1-score   support

           0       0.79      0.73      0.76      1867
           1       0.75      0.81      0.78      1883

    accuracy                           0.77      3750
   macro avg       0.77      0.77      0.77      3750
weighted avg       0.77      0.77      0.77      3750



In [284]:
from eli5.lime import TextExplainer

te = TextExplainer(random_state=42)

In [285]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

In [286]:
from xgboost import XGBClassifier

In [287]:
xgbclassifier = XGBClassifier(**study.best_trial.params)

In [288]:
xgbclassifier.fit(trainx, trainy)

/Users/mayorao/opt/miniconda3/envs/bertnlp/lib/python3.8/site-packages/xgboost/sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifier(alpha=1.6837667313044715e-05, base_score=0.5, booster='dart',
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              eta=0.9995441483545477, gamma=8.685419715245789e-08, gpu_id=-1,
              grow_policy='depthwise', importance_type='gain',
              interaction_constraints='', lambda=4.3445606132944827e-07,
              learning_rate=0.999544144, max_delta_step=0, max_depth=9,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, normalize_type='tree',
              num_parallel_tree=1, random_state=0,
              rate_drop=2.53345870065096e-08, reg_alpha=1.68376664e-05,
              reg_lambda=4.34456069e-07, sample_type='uniform',
              scale_pos_weight=1, skip_drop=7.839688797876259e-07, ...)

In [289]:
predicted_sentiments = np.where(xgbclassifier.predict(testx)>=0.5,1,0)

In [290]:
print(classification_report(testy, predicted_sentiments))

              precision    recall  f1-score   support

           0       0.84      0.82      0.83      1867
           1       0.83      0.85      0.84      1883

    accuracy                           0.83      3750
   macro avg       0.83      0.83      0.83      3750
weighted avg       0.83      0.83      0.83      3750



In [291]:
predict_pipeline = Pipeline([('vectorizer', tf_vectorizer),('model', xgbclassifier)])

In [292]:
predict_pipeline.predict_proba(["This is an interesting movie"])

array([[0.2660128, 0.7339872]], dtype=float32)

In [306]:
predict_pipeline.predict_proba([df['review'][2]])

array([[0.9977201 , 0.00227991]], dtype=float32)

In [305]:
df['review'][2]

'im in iraq right now doing a job that gives plenty of time for watching movies we also have access to plenty of pirated movies this gem came along with 11 other movies and this is easily the worst ive seen in a long time ive seen a few other reviews that claim this movie doesnt take itself too seriously but really i think thats a cover up for the fact that its horrible its not tongue in cheek the writers really thought they were improving on the movie blade this movie is just one notch above vampire assassin which if you havent seen i recommend at least that movie is so unbelievably bad that youll laugh harder than you thought possible this is right at that cusp of no redeeming qualities what so ever from the bad acting to cliché visual ie opening credits to the adobe premier special effects they couldnt even get blanks for the guns which may have to do with where the movie was filmed but if youre going to use effects make them close to accurate as for the cast it seems like they just

In [293]:
te.fit(df['review'][2], predict_pipeline.predict_proba)
te.show_prediction(target)

In [294]:
misclassified_index = np.argwhere(testy != predicted_sentiments)

In [295]:
testx

<3750x4542156 sparse matrix of type '<class 'numpy.float64'>'
	with 1202499 stored elements in Compressed Sparse Row format>

In [297]:
misclassified = testx_orig.values[misclassified_index]

In [298]:
misclassified.shape

(622, 1)

In [299]:
misclassified = misclassified.reshape(-1,)

In [300]:
misclassification_report = pd.DataFrame(zip(misclassified,testy[misclassified_index].reshape(-1,)), columns=["review","sentiment"])

In [301]:
misclassification_report

,review,sentiment
0,the movie looked like a walkthrough for immoral study most likely i never got much involved with the burning need of the female artist to immortalize male nudes and thus all that fuss about now who drew this penis? sounded a bit gratuitous dialogues in this movie are rather dreadful albeit visually this movie got its moments i almost dig it when tassi got into painting a mental picture but then movie weered back onto penises highly recommended to those who has not seen one in a while,1
1,my wife is a teacher and she is very familiar with the story having read it to several of her classes it never sounded all that interesting to me though and i bought the dvd figuring this would be a movie that wouldnt really be up my alleybr /br /the first half of the movie has a lot of setup and i found myself thinking that i was right it starts off a bit slow and i have to admit that i was a little bit bored but curious enough to stay with it boy am i glad i did because this ended up being a very satisfying and rewarding movie i would most certainly watch this againbr /br /the casting was very good since i havent read the book i cant vouch for accuracy but i have to say that jon voight was truly delightful you liked the characters you were supposed to like hated the ones you were supposed to hate and laughed at the ones that were supposed to be funnybr /br /i can see how some folks might not like this movie it is tedious at times especially in the beginning all the flashbacks can be distracting though they are essential to the story once the story starts to come together at the end though i think youre paid back in spades for your patience when all is said and done i think this is a very good movie 8/10br /br /,1
2,is miike like chabrol alternating art with dreck sometimes confusing the two? does he match the fifty/fifty rate some claim for chabrol? do we see here too much or too little miike? i dont know i dont know i dont know but i could easily fill ten lines just citing chakushin aris one missed calls steals allusions and clichés others here will hit on most of them though not perhaps ochiais saimin hypnosis for the sleazy telecast taunting things supernatural only once when yumi takes the zombielike mother in her arms did one missed call startle me a littlebr /br /one other point of interest: renji ishibashi looks like christopher walken as the detective ishibashi in auditions abusive back story and as dead or alives boss aoki of the vat of excrement drowning and more may be the most convincingly evil character actor ive ever seen so here hes a cop,0
3,in the comic modesty is strong alexandra staden who plays modesty blaise looks more like an anorectic fashion model she does not either have the moral or personality that modesty have in the comics modesty would never give a woman an advice to show more skin to earn more money i cannot see any similarities with my comic books with modesty and this movie its like a mission impossible movie would be about ethan hunt locked in the detention room in high school talking with the janitor about when he went to junior high school and hunt would have been played by dj qualls in road trip soo if you are an modesty fan do not see the movie you will just get angry if do not know much about the modesty comics rent an other movie do not wast your time with this onei cannot understand how quentin tarantino can put his name on it i will ask for a refund at my dvd rent store tomorrow,0
4,this movie is a lot better than the asylums version mainly its war of the worlds the tripods look pretty cool but their walking and deaths could have been better the action scenes were really cool walking walkingwalkingwalking oh my god stop walking please or im going to kill myself the thunder child scene was my favorite sequence mainly because a ship rammed bunch of tripods good movie i recommend it for people ho have read the book the music is awesome and the directors cut looks pretty coolbr /br /p

In [302]:
misclassification_report['review'][0]

'the movie looked like a walkthrough for immoral study most likely i never got much involved with the burning need of the female artist to immortalize male nudes and thus all that fuss about now who drew this penis? sounded a bit gratuitous dialogues in this movie are rather dreadful albeit visually this movie got its moments i almost dig it when tassi got into painting a mental picture but then movie weered back onto penises highly recommended to those who has not seen one in a while'

In [310]:
predict_pipeline.predict_proba([misclassification_report['review'][0]])

array([[0.79821235, 0.20178765]], dtype=float32)

In [313]:
te.fit(misclassification_report['review'][617], predict_pipeline.predict_proba)
te.show_prediction(targets=[0,1],target_names=['neg', 'pos'])

In [312]:
te.fit("This is the best movie ever",  predict_pipeline.predict_proba)
te.show_prediction(targets=[0,1],target_names=['neg', 'pos'])

In [ ]:
eli5.formatters.html.format_as_html(te.explain_prediction(targets=[0,1],target_names=['neg', 'pos']))

# BERT based Classification

In [26]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertForSequenceClassification
from transformers import AdamW
from collections import defaultdict

In [27]:
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'

In [28]:
model = BertForSequenceClassification.from_pretrained(PRE_TRAINED_MODEL_NAME, num_labels=1)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [30]:
if os.path.exists('bert_movie_classifier.mdl'):
    print ("Loading exiting model")
    model.load_state_dict(torch.load('bert_movie_classifier.mdl'))

Loading exiting model


In [34]:
# class BinaryReviewClassifier(nn.Module):
    
#     def __init__(self):
#         self.model = BertForSequenceClassification.from_pretrained(PRE_TRAINED_MODEL_NAME, num_labels=1)
#         self.sigmoid = 

In [35]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [36]:
tokens = tokenizer.tokenize("This is an interesting movie")

In [37]:
tokens

['This', 'is', 'an', 'interesting', 'movie']

In [38]:
token_ids = tokenizer.convert_tokens_to_ids(tokens)

In [39]:
token_ids

[1188, 1110, 1126, 5426, 2523]

In [40]:
class MovieReviewDataset(Dataset):

    def __init__(self, reviews, targets, tokenizer, max_len):
        self.reviews = reviews
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, item):
        review = str(self.reviews[item])
        target = self.targets[item]
        encoding = self.tokenizer.encode_plus(
          review,
          add_special_tokens=True,
          max_length=self.max_len,
          return_token_type_ids=False,
          padding='max_length',
          return_attention_mask=True,
          return_tensors='pt',
            truncation=True
        )
        return {
          'review_text': review,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'targets': torch.tensor(target, dtype=torch.float)
        }

In [41]:
def create_dataloader(reviews, target, tokenizer, max_len, batch_size):
    ds = MovieReviewDataset(
        reviews=reviews,
        targets=target,
        tokenizer=tokenizer,
        max_len=max_len
    )
    return DataLoader(
        ds,
        batch_size=batch_size
    )

In [42]:
# train_bert_tokens_input = tokenizer(trainx_orig.values.tolist(), return_tensors='pt', padding=True, truncation=True)
# train_input_ids = train_bert_tokens_input['input_ids']
# train_attention_mask = train_bert_tokens_input['attention_mask']

In [43]:
# trainx_orig.values.shape

In [44]:
training_dataloader = create_dataloader(trainx_orig.values.tolist(), trainy.tolist(), tokenizer, 150, 25)

In [45]:
validation_dataloader = create_dataloader(testx_orig.values.tolist(), testy.tolist(), tokenizer, 150, 25)

In [223]:
EPOCHS = 10
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
lossfn = nn.BCEWithLogitsLoss(reduction="sum")

In [224]:
epoch_loss = {}
for i in tqdm(range(EPOCHS), position=0, leave=True):
    loss_history = defaultdict(list)
    model.train()
    for data in tqdm(training_dataloader, position=0, leave=True, desc=f"Training: Epoch: {i+1}"):
        outputs = model(
          input_ids = data['input_ids'],
          attention_mask = data['attention_mask']
        )

        loss = lossfn(torch.squeeze(outputs[0]), data['targets'])
        loss_history['train'].append(loss.item())
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    model.eval()
    val_outputs = []
    for data in tqdm(validation_dataloader, position=0, leave=True, desc=f"Validation: Epoch: {i+1}"):
        outputs = model(
          input_ids = data['input_ids'],
          attention_mask = data['attention_mask']
        )
        predictions = torch.squeeze(torch.sigmoid(outputs[0])).detach().cpu().numpy().tolist()
        # normalized_preds = np.where(np.array(predictions)>=0.5,1,0)
        # y = data['targets'].detach().cpu().numpy()
        
        
        val_outputs += predictions
    normalized_preds = np.where(np.array(val_outputs)>=0.5,1,0)
    accuracy = accuracy_score(testy,normalized_preds)

    auc_score = roc_auc_score(testy, normalized_preds)
    print(f"Validation Accuracy is: {accuracy} and AUC is: {auc_score}")
    loss_history['validation'].append(auc_score)
    epoch_loss[i] = loss_history


Training: Epoch: 2:   0%|          | 0/850 [00:00<?, ?it/s]

Validation Accuracy is: 0.8685333333333334 and AUC is: 0.867524893314367


Training: Epoch: 3:   0%|          | 0/850 [00:00<?, ?it/s]

Validation Accuracy is: 0.8784 and AUC is: 0.8781152204836415


Training: Epoch: 4:   0%|          | 0/850 [00:00<?, ?it/s]

...

Validation Accuracy is: 0.8768 and AUC is: 0.8760597439544808


Training: Epoch: 10:   0%|          | 0/850 [00:00<?, ?it/s]

Validation Accuracy is: 0.8528 and AUC is: 0.8515433854907539


100%|██████████| 10/10 [27:28:43<00:00, 9892.36s/it] 

Validation Accuracy is: 0.8754666666666666 and AUC is: 0.8748079658605975


In [225]:
torch.save(model.state_dict(), 'bert_movie_classifier.mdl')

In [227]:
{i:(np.average(epoch_loss[i]['train']), np.average(epoch_loss[i]['validation'])) for i in epoch_loss}

{0: (9.47308171636918, 0.867524893314367),
 1: (5.12582640171051, 0.8781152204836415),
 2: (2.78663356198984, 0.8636344238975818),
 3: (1.7614505276215426, 0.8780654338549075),
 4: (1.2422575635419173, 0.8668563300142248),
 5: (0.8884004055840128, 0.8660881934566145),
 6: (0.6991691917378237, 0.8632147937411095),
 7: (0.6434592808563919, 0.8760597439544808),
 8: (0.6042927231935455, 0.8515433854907539),
 9: (0.606832971586024, 0.8748079658605975)}

In [31]:
import numpy as np

In [32]:
def predict(sample_review, max_len=150):
    input_ids = []
    attention_masks = []
    if not isinstance(sample_review, list):
        raise Exception("Required input is list")
    for item in sample_review:
        encoding =tokenizer.encode_plus(
                  item,
                  add_special_tokens=True,
                  max_length=max_len,
                  return_token_type_ids=False,
                  padding='max_length',
                  return_attention_mask=True,
                  return_tensors='pt',
                    truncation=True
                )
#         print (encoding)
        input_ids.append(encoding['input_ids'])
        attention_masks.append(encoding['attention_mask'])
    return np.where(torch.sigmoid(model(torch.cat(input_ids, dim=0), torch.cat(attention_masks, dim=0))[0]).detach().cpu().numpy() >= 0.5, 1, 0).flatten()

In [46]:
predict(["This is an interesting movie", "This is the worst movie"])


array([1, 0])

In [51]:
val_outputs = []
for data in tqdm(validation_dataloader, position=0, leave=True):
    outputs = model(
      input_ids = data['input_ids'],
      attention_mask = data['attention_mask']
    )
    predictions = torch.squeeze(torch.sigmoid(outputs[0])).detach().cpu().numpy().tolist()
    # normalized_preds = np.where(np.array(predictions)>=0.5,1,0)
    # y = data['targets'].detach().cpu().numpy()


    val_outputs += predictions
normalized_preds = np.where(np.array(val_outputs)>=0.5,1,0)    

100%|██████████| 150/150 [09:39<00:00,  3.87s/it]


In [52]:
print(classification_report(testy, normalized_preds))

              precision    recall  f1-score   support

           0       0.91      0.83      0.87      1850
           1       0.84      0.92      0.88      1900

    accuracy                           0.88      3750
   macro avg       0.88      0.87      0.88      3750
weighted avg       0.88      0.88      0.88      3750

